In [1]:
import pandas as pd
import numpy as np
from dataset_dir import stata_dir
from dataset_dir import excel_dir
import os
import seaborn as sns
from pandas_profiling import ProfileReport
import datetime
date_today= datetime.date.today()

In [2]:
pd.io.formats.excel.ExcelFormatter.header_style = None

In [3]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [4]:
r=pd.read_stata(stata_dir()+"Teamup_Women_Dataset_cleanv4.dta",convert_categoricals=False)

In [5]:
# r=r.loc[(r['level1']=='nigeria') & (r['consent']==1) & (r['result']==1) ]
r.shape

(1530, 10400)

In [7]:
# r.result

In [6]:
strategies={1:'Withdrawal',9:'Rhythm',5:'Herbs'}


In [53]:
x=6;y=7
v="hello%s%s"%(x,y)
print(v)

hello67


In [14]:
def replace_yesno(col_name,m):
    m[col_name]=m[col_name].replace(1,"Yes")
    m[col_name]=m[col_name].replace(0,"No")
    return m

def replace_degree(col_name,m):
    m[col_name]=m[col_name].replace(1,"All the time")
    m[col_name]=m[col_name].replace(2,"Most of the time")
    m[col_name]=m[col_name].replace(3,"Half of the time")
    m[col_name]=m[col_name].replace(4,"Rarely")
    m[col_name]=m[col_name].replace(5,"Once")
    
    
    


# def replaceables(m):
#     replace_yesno('used a strategy in 6 mth q209b',m)
#     replace_yesno('used a strategy in 1 mth q209a',m)
#     replace_yesno('used withdrawal in last 1 mth q211a_1',m)
#     replace_yesno('used withdrawal in last 6 mth q215_1',m)
#     replace_yesno('still using withdrawal[1st selection] q220d_1',m)
#     replace_yesno('still using withdrawal[2nd selection] q221d_1',m)
#     replace_yesno('probe: used withdrawal in last 1 month q211c_1',m) 
#     replace_yesno('probe: used withdrawal in last 6 month q216c_1',m) 
#     replace_yesno('used withdrawal with other method q220g_1[1st selection]',m)
#     replace_yesno('used withdrawal with other method q221g_1[2nd selection]',m)
#     replace_yesno('when you stopped withdrawal, did you start another strat? q220f_1',m)
#     replace_yesno('when you stopped withdrawal, did you start another strat? q221f_1',m)
#     replace_yesno('ever used withdrawal q224_1',m)
#     replace_degree('how often used withdrawal in 6 months q220a_1',m)
#     replace_degree('how often used withdrawal with other strategy q220i_1',m)
#     return m



In [25]:
strategies={1:'Withdrawal',2:'Abstinence_Rhythm',3:'counting_plus',4:'concoctions',
            5:'herbs',6:'Rings_amulets_charms_and_waist_strings',7:'Lactational_Amenorrhea_method',
            8:'Non_LAM_breastfeeding',9:'Standard_days_method',10:'Cervical_mucus_strategy',
            11:'basal_body_temperature_strategy',21:'Douching',22:'Drinking_a_cold_drink',23:'Taking_a_cold_shower',24:'Masturbation',25:'Sex_Toys_Aids',26:'Intercrural_sex',27:'Oral_sex',28:'Anal_sex'}
check_population=[]
def new_profiles(data):
    for k,v in zip(data.keys(),data.values()):
        k=str(k)
        select_cols=['q221i_%s'%k,'q221a_%s'%k,'q221g_%s'%k,'q220f_%s'%k,'q221f_%s'%k,'q221d_%s'%k,'q216c_%s'%k,'q209a','q209b','q211a_%s'%k,'q215_%s'%k,'q220d_%s'%k,'q220a_%s'%k,'q220g_%s'%k,'q211c_%s'%k,'q220i_%s'%k,'q224_%s'%k,'resp_select'] 
        def replaceables(m):
            replace_yesno('Used any strategy in the last 2-6 months (q209b)',m)
            replace_yesno('Used any strategy in the last 1 month (q209a)',m)
            replace_yesno('Used %s in the last 1 month (q211a_%s)'%(v,k),m)
            replace_yesno('Used %s in last 2-6 months (q215_%s)'%(v,k),m)
            replace_yesno('Still using %s[1st selection] (q220d_%s)'%(v,k),m)
            replace_yesno('Still using %s[2nd selection] (q221d_%s)'%(v,k),m)
            replace_yesno('Probe: used %s in the last 1 month (q211c_%s)'%(v,k),m) 
            replace_yesno('Probe: used %s in last 2-6 months (q216c_%s)'%(v,k),m) 
            replace_yesno('Used %s with another strategy[1st selection] (q220g_%s)'%(v,k),m)
            replace_yesno('Used %s with another strategy[2nd selection] (q221g_%s)'%(v,k),m)
            replace_yesno('When you stopped using %s, did you start using another strategy? (q220f_%s)'%(v,k),m)
            replace_yesno('When you stopped using %s, did you start using another strategy? (q221f_%s)'%(v,k),m)
            replace_yesno('From any time in the past, ever used %s (q224_%s)'%(v,k),m)
            replace_degree('How often respondent used %s in the last 2-6 months [1st selection] (q220a_%s)'%(v,k),m)
            replace_degree('How often respondent used %s in the last 2-6 months [2nd selection] (q221a_%s)'%(v,k),m)
            replace_degree('How often respondent used %s with another strategy [1st selection] (q220i_%s)'%(v,k),m)
            replace_degree('How often respondent used %s with another strategy [2nd selection] (q221i_%s)'%(v,k),m)
            return m        
        
        
        
        
        
        
        
        def gen_df(n):
            holder=pd.DataFrame({'id':n['resp_select'],
                'Used any strategy in the last 1 month (q209a)':n['q209a'],
                             'Used any strategy in the last 2-6 months (q209b)':n['q209b'],
                             'Used %s in the last 1 month (q211a_%s)'%(v,k):n['q211a_%s'%k],
                             'Used %s in last 2-6 months (q215_%s)'%(v,k):n['q215_%s'%k],
                             'Still using %s[1st selection] (q220d_%s)'%(v,k):n['q220d_%s'%k],
                             'Still using %s[2nd selection] (q221d_%s)'%(v,k):n['q221d_%s'%k],
                             'When you stopped using %s, did you start using another strategy? (q220f_%s)'%(v,k):n['q220f_%s'%k],
                             'When you stopped using %s, did you start using another strategy? (q221f_%s)'%(v,k):n['q221f_%s'%k],

                            'Probe: used %s in the last 1 month (q211c_%s)'%(v,k):n['q211c_%s'%k],
                                 'Probe: used %s in last 2-6 months (q216c_%s)'%(v,k):n['q216c_%s'%k],
                             'How often respondent used %s in the last 2-6 months [1st selection] (q220a_%s)'%(v,k):n['q220a_%s'%k],
                             'How often respondent used %s in the last 2-6 months [2nd selection] (q221a_%s)'%(v,k):n['q221a_%s'%k],
                             'Used %s with another strategy[1st selection] (q220g_%s)'%(v,k):n['q220g_%s'%k],
                             'Used %s with another strategy[2nd selection] (q221g_%s)'%(v,k):n['q221g_%s'%k],
                             'How often respondent used %s with another strategy [1st selection] (q220i_%s)'%(v,k):n['q220i_%s'%k],
                             'How often respondent used %s with another strategy [2nd selection] (q221i_%s)'%(v,k):n['q221i_%s'%k],
                             'From any time in the past, ever used %s (q224_%s)'%(v,k):n['q224_%s'%k]   
                                })
            return holder
   # ------------------------------------------------------------------------------------------------     
        #                                     CONSISTENT EXCLUSIVE or CONSISTENT USER
   # ------------------------------------------------------------------------------------------------  
        n=r.loc[
        ((r['q211a_%s'%k]==1.0) |(r['q211c_%s'%k]==1.0) | (r['q215_%s'%k]==1.0) | (r['q216c_%s'%k]==1.0)) #respondent used target strategy in last 1-6 months
        &
        ((r['q220a_%s'%k]==1.0) | (r['q220a_%s'%k]==2.0) | (r['q221a_%s'%k]==1.0) | (r['q221a_%s'%k]==2.0))#how often you used withdrawal in last 6 months
        &
        ((r['q220f_%s'%k]!=1.0) & (r['q221f_%s'%k]!=1.0)) #didnt switch to another strategy   
        # &
        # (r['q220a_%s'%k].notnull())
        &
        ((r['q220g_%s'%k]==0.0) | (r['q221g_%s'%k]==0.0)) #used withdrawal with other method
        ][select_cols]

        ce=gen_df(n)
   # ------------------------------------------------------------------------------------------------     
        #                                     INCONSISTENT EXCLUSIVE or INCONSISTENT USER
   # ------------------------------------------------------------------------------------------------    
        n=r.loc[
        ((r['q211a_%s'%k]==1.0) |(r['q211c_%s'%k]==1.0) | (r['q215_%s'%k]==1.0) | (r['q216c_%s'%k]==1.0)) #respondent used target strategy in last 1-6 months
        &
        ((r['q220a_%s'%k]==3.0) | (r['q220a_%s'%k]==4.0) |(r['q220a_%s'%k]==5.0) | (r['q221a_%s'%k]==3.0) | (r['q221a_%s'%k]==4.0) | (r['q221a_%s'%k]==5.0))#how often you used withdrawal in last 6 months
        &
        ((r['q220f_%s'%k]!=1.0) & (r['q221f_%s'%k]!=1.0)) #didnt switch to another strategy   
        # &
        # (r['q220a_%s'%k].notnull())
        &
        ((r['q220g_%s'%k]==0.0) | (r['q221g_%s'%k]==0.0)) #used withdrawal with other method
        ][select_cols]

        ie=gen_df(n)
   # ------------------------------------------------------------------------------------------------     

#                               CONSISTENT CONCURRENT or CONSURRENT USER
# --------------------------------------------------------------------------------------------

        n=r.loc[
        ((r['q211a_%s'%k]==1.0) |(r['q211c_%s'%k]==1.0) | (r['q215_%s'%k]==1.0) | (r['q216c_%s'%k]==1.0)) #respondent used withdrawal in last 6 months
        # &
        # ((r['q220a_%s'%k]==1.0) | (r['q220a_%s'%k]==2.0) | (r['q221a_%s'%k]==1.0) | (r['q221a_%s'%k]==2.0) | (r['q221a_%s'%k].isna()) )#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
        &
        ((r['q220g_%s'%k]==1.0) | (r['q221g_%s'%k]==1.0)) #used strategy with other strategy
        &
        ((r['q220d_%s'%k]==1.0) | (r['q221d_%s'%k]==1.0)) #Still using strategy

        
#         &
#         ((r['q220i_%s'%k]==1.0) | (r['q220i_%s'%k]==2.0) | (r['q221i_%s'%k]==1.0) | (r['q221i_%s'%k]==2.0) | (r['q221a_%s'%k].isna()))#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
 
        ][select_cols]

        cc=gen_df(n)
        
   # ------------------------------------------------------------------------------------------------     
        #                                     CONSISTENT SPORADIC
   # ------------------------------------------------------------------------------------------------  

        n=r.loc[
        ((r['q211a_%s'%k]==1.0) |(r['q211c_%s'%k]==1.0) | (r['q215_%s'%k]==1.0) | (r['q216c_%s'%k]==1.0)) #respondent used withdrawal in last 6 months
        &
        ((r['q220a_%s'%k]==1.0) | (r['q220a_%s'%k]==2.0) | (r['q221a_%s'%k]==1.0) | (r['q221a_%s'%k]==2.0) )#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
        &
        ((r['q220i_%s'%k]==3.0) | (r['q220i_%s'%k]==4.0) | (r['q221i_%s'%k]==3.0) | (r['q221i_%s'%k]==4.0) )#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
        #  &
        # ((r['q221i_%s'%k].notna() & (r['q221i_%s'%k].notna()))
        ][select_cols]

        cs=gen_df(n)
   # ------------------------------------------------------------------------------------------------     
        #                                     INCONSISTENT SPORADIC
   # ------------------------------------------------------------------------------------------------ 
        n=r.loc[
        ((r['q211a_%s'%k]==1.0) |(r['q211c_%s'%k]==1.0) | (r['q215_%s'%k]==1.0) | (r['q216c_%s'%k]==1.0)) #respondent used withdrawal in last 6 months
        &
        ((r['q220a_%s'%k]==3.0) | (r['q220a_%s'%k]==4.0) | (r['q221a_%s'%k]==3.0) | (r['q221a_%s'%k]==4.0) )#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
        &
        ((r['q220i_%s'%k]==3.0) | (r['q220i_%s'%k]==4.0) | (r['q221i_%s'%k]==3.0) | (r['q221i_%s'%k]==4.0) )#respondent uses withdrawal with another method ALL OF THE TIME or MOST OF THE TIME
        #  &
        ][select_cols]

        _is=gen_df(n)
   # ------------------------------------------------------------------------------------------------       
                #                                        DISCONTINUED or STOPPERS
   # ------------------------------------------------------------------------------------------------    
        
        n=r.loc[
        ((r['q220d_%s'%k]==0) | (r['q221d_%s'%k]==0)) #stopped using target strategy
        &
        ((r['q220f_%s'%k]==0) | (r['q221f_%s'%k]==0)) #didnt switch to another strategy    
         &
        ((r['q211a_%s'%k]==0) | (r['q211c_%s'%k]==0)) #didnt use target strategy in the last 1 month
        &
        (r['q209a']==0)
        

        ][select_cols]

        d=gen_df(n)
   # ------------------------------------------------------------------------------------------------     
         #                                        SWITCHED or SWITCHERS
   # ------------------------------------------------------------------------------------------------     
                                                                                                 
       
        n=r.loc[

        ((r['q220f_%s'%k]==1) | (r['q221f_%s'%k]==1)) #used target strategy and switched to another strategy after stopping
        &
        ((r['q215_%s'%k]==1) | (r['q216c_%s'%k]==1)) #used target strategy in last 2-6 months   
        
        ][select_cols]
        s=gen_df(n)

# --------------------------------------------------------------------------------------------
        m=pd.concat([cc,cs,_is],axis=0)

        writer = pd.ExcelWriter(excel_dir()+"profile_%s%s.xlsx"%(v,date_today), engine='xlsxwriter')
#         writer2 = pd.ExcelWriter(excel_dir()+"profile_%s_16_11_2022.xlsx"%v, engine='xlsxwriter')
        
        
        #summary
        def get_summary(x,profile_name):
            j=[x.shape[0],r.shape[0]-x.shape[0],r.shape[0]]
            jj=[(x.shape[0]/r.shape[0])*100,((r.shape[0]-x.shape[0])/r.shape[0])*100,100]
            g=pd.DataFrame({'N':j,'%':jj},index=['%s | Yes'%profile_name,'No','Total'])
            return g
        def meta(x,profile_name):
            n_yes=x.shape[0]
            return n_yes
        
        s0=get_summary(ce,"Consistent User")
        s0.to_excel(writer,sheet_name = 'Summary',startrow=0)
        check_population.append(meta(ce,"Consistent User"))
        

        s5=get_summary(ie,"Inconsistent User")
        s5.to_excel(writer,sheet_name = 'Summary',startrow=5)
        check_population.append(meta(ie,"Consistent User"))
        # s10=get_summary(m,"concurrent user")
        # s10.to_excel(writer,sheet_name = 'Summary',startrow=10)
        s10=get_summary(cc,"Concurrent User")
        s10.to_excel(writer,sheet_name = 'Summary',startrow=10)
        check_population.append(meta(cc,"Consistent User"))
        # s20=get_summary(cs,"consistent sporadic")
        # s20.to_excel(writer,sheet_name = 'Summary',startrow=20)
        # s25=get_summary(_is,"inconsistent sporadic")
        # s25.to_excel(writer,sheet_name = 'Summary',startrow=25)
        s15=get_summary(d,"Stopper")
        s15.to_excel(writer,sheet_name = 'Summary',startrow=15)
        check_population.append(meta(d,"Consistent User"))
        s20=get_summary(s,"Switcher")
        s20.to_excel(writer,sheet_name = 'Summary',startrow=20)
        check_population.append(meta(s,"Consistent User"))
        
     
     

        
        replaceables(ce)
        ce.to_excel(writer,sheet_name = 'Consistent User', index=False)
        replaceables(ie)
        ie.to_excel(writer,sheet_name = 'Inconsistent User', index=False)
        # replaceables(m)
        # m.to_excel(writer,sheet_name = 'Concurrent User', index=False)
        replaceables(cc)
        cc.to_excel(writer,sheet_name = "Concurrent User", index=False)
        # replaceables(cs)
        # cs.to_excel(writer,sheet_name = "Consistent Sporadic", index=False)
        # replaceables(_is)
        # _is.to_excel(writer,sheet_name = "Inconsistent Sporadic", index=False)
        replaceables(d)
        d.to_excel(writer,sheet_name = 'Stopper', index=False)
        replaceables(s)
        s.to_excel(writer,sheet_name = 'Switcher', index=False)
        
        workbook  = writer.book
        worksheet = writer.sheets["Summary"]
        data_format1 = workbook.add_format({'bg_color': '#b8fcff'})
        data_format2 = workbook.add_format({'align': 'right'})
        worksheet.set_column('A:A',30,cell_format=data_format2)
        for i in range(0,22,5):
            worksheet.set_row(i, cell_format=data_format1)
        # worksheet.set_row(row, cell_format=data_format1)
        l=["Consistent User","Inconsistent User","Concurrent User",'Stopper','Switcher']
        for i in l:
            worksheet2 = writer.sheets[i]
            data_format3 = workbook.add_format({'bg_color': '#b8fcff','bold':True,'text_wrap': True,'border':1})
            # data_format2 = workbook.add_format({'align': 'right'})
            worksheet2.set_row(0,cell_format=data_format3)
            worksheet2.set_column('A:Z',15)
        
        #summary

        writer.save()
        print(f"Profiles for {v} Generated!")
        # os.system("start EXCEL.EXE "+excel_dir()+"profile_%s%s.xlsx"%(v,date_today))
        
new_profiles(strategies)

Profiles for Withdrawal Generated!
Profiles for Abstinence_Rhythm Generated!
Profiles for counting_plus Generated!
Profiles for concoctions Generated!
Profiles for herbs Generated!
Profiles for Rings_amulets_charms_and_waist_strings Generated!
Profiles for Lactational_Amenorrhea_method Generated!
Profiles for Non_LAM_breastfeeding Generated!
Profiles for Standard_days_method Generated!
Profiles for Cervical_mucus_strategy Generated!
Profiles for basal_body_temperature_strategy Generated!
Profiles for Douching Generated!
Profiles for Drinking_a_cold_drink Generated!
Profiles for Taking_a_cold_shower Generated!
Profiles for Masturbation Generated!
Profiles for Sex_Toys_Aids Generated!
Profiles for Intercrural_sex Generated!
Profiles for Oral_sex Generated!
Profiles for Anal_sex Generated!


End!

Check Population

In [31]:
check_population=np.array(check_population)
check_population.sum()

188